In [1]:
# Bank id

b_id = 29
#bc_ids = [17, 9, 29, 26, 50, 4, 5, 13, 64]

In [2]:
import sys

sys.path.append('../..')

In [3]:
def rec_dict_print(a_dict, level=1):
    keys = a_dict.keys()
    keys.sort()
    for k in keys:
        if type(a_dict[k]) is dict:
            print "".join(["-" for i in range(level)]) + k
            rec_dict_print(a_dict[k], level + 2)
        else:
            print "".join(["-" for i in range(level)]) + k

In [4]:
%matplotlib inline
import dill

loaded_data = None

with open('../precomputed_sims/data.pkl', 'rb') as f:
    loaded_data = dill.load(f)

In [5]:
rec_dict_print(loaded_data)

-N
-credit
---bc_ids
---bc_subsets_indexes
---copula
---default_times
-randomization
---distrib
-----loc
-----scale
---maturity
---zetas
-underlyings


In [6]:
# Instead of taking 365 standard days or 252 trading days
# in order to get some easy computations for the eqty and df time grids
# I chose to take 360 days of tradings

step = 1/360.
delta = 5*step

maturity = loaded_data["randomization"]["maturity"]

print "Maturity = %s years" % maturity

Maturity = 5.0 years


In [7]:
import warnings

warnings.filterwarnings("error")

# Discount

In [8]:
from finance.discountfactor import ConstantRateDiscountFactor 

r = 0.02
discount = ConstantRateDiscountFactor(r)

# Underlying

In [9]:
udlyings = loaded_data["underlyings"]

print "Maximum number of paths: %i" % len(udlyings)

gbm0 = udlyings[0]

kappa = gbm0.drifts[0][0]
sigma = gbm0.vols[0][0]
print "kappa = %s, sigma = %s" % (kappa, sigma)

time_grid = gbm0.time

Maximum number of paths: 20000
kappa = 0.12, sigma = 0.2


# Derivative

In [10]:
derivatives_nb = 1

In [11]:
from finance.products.european.swap import (
    SwapContract,
)

swap_delta = 0.25

swap_dates = SwapContract.generate_payment_dates(0, maturity, swap_delta)
swap = SwapContract(gbm0, discount, swap_dates)

price_0 = swap.price(0., incl_next_coupon=True)

print swap
print "\nPrice swap at t=0 without 1st coupon = %s" % price_0

Swap contract of maturity T = 5 years, over S^0 with strike K = 134.306, paying at {0.00, 0.25, 0.50, 0.75, 1.00, 1.25, 1.50, 1.75, 2.00, 2.25, 2.50, 2.75, 3.00, 3.25, 3.50, 3.75, 4.00, 4.25, 4.50, 4.75, 5.00}

Price swap at t=0 without 1st coupon = -1.13686837722e-13


# Exposure

In [12]:
from risk.exposures import EuropeanVaRGeomBrownianExposure

quantile_im = 0.65
exposure = EuropeanVaRGeomBrownianExposure(swap, discount, kappa, sigma)

In [13]:
from ccp.covertworule import LCHEmirCoverTwo

cover2rule = LCHEmirCoverTwo()

The history saving thread hit an unexpected error (OperationalError('database is locked',)).History will not be written to the database.


In [14]:
from risk.basel.eee import BlackScholesSwapVaREffExpectExposure

eee = BlackScholesSwapVaREffExpectExposure(swap)

# Indexes stuffs

## Ids of $B$ and $C$

In [15]:
copula = loaded_data["credit"]["copula"]

bc_subsets_indexes = loaded_data["credit"]["bc_subsets_indexes"]

#bc_ids = [26, 29, 17, 50, 13, 4, 5, 9, 64]
#bc_positions = [-0.36, -0.44, 0.69, 0.34, -0.05, 0.23, 0.09, -0.46, -0.04]
#Spreads [1053, 367, 176, 73, 61, 56, 52, 45, 108]

bc_ids = [17, 9, 29, 26, 50, 4, 5, 13, 64]
bc_positions = [0.69, -0.46, -0.44, -0.36, 0.34, 0.23, 0.09, -0.05, -0.04]

# Counterparties id
c_ids = list(set(bc_ids) - set([b_id]))

print "Bank id: %s" % b_id
print "Counterparties id: %s (nb = %s)" % (c_ids, len(c_ids))

Bank id: 29
Counterparties id: [64, 4, 5, 9, 13, 17, 50, 26] (nb = 8)


In [16]:
obligors_nb = len(copula.subsets[bc_subsets_indexes[-1]][0])

print "Obligor numbers: %s" % obligors_nb

Obligor numbers: 125


## Subsets indexes of the copula for $B$ and $C$

In [17]:
b_subsets_indexes = copula.get_indexes_including(b_id)

c_subsets_indexes = []
for c_id in c_ids:
    tmp = copula.get_indexes_including(c_id)
    c_subsets_indexes += tmp
    
b_subsets_indexes.sort()
c_subsets_indexes.sort()
    
print "Subsets that generated default times: %s\n" % bc_subsets_indexes
print "Bank subsets indexes: %s\n" % b_subsets_indexes

tmp_c_subsets_indexes = list(set(c_subsets_indexes))
tmp_c_subsets_indexes.sort()
print "Counterparties subsets indexes: %s" % tmp_c_subsets_indexes

Subsets that generated default times: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129]

Bank subsets indexes: [29, 126, 127, 128, 129]

Counterparties subsets indexes: [4, 5, 9, 13, 17, 26, 50, 64, 125, 126, 127, 128, 129]


# Portfolio construction

In [18]:
import numpy as np

p_fixed = 1.
strike = swap.strike

delta_times = swap.delta_time
discount_factors = [discount(t) for t in swap.pillars[1:]]

delta_beta_sum = np.dot(delta_times, discount_factors)

notional = p_fixed / (strike*delta_beta_sum)

print "Notional on the swap: %s" % notional

Notional on the swap: 0.0015687485053


In [19]:
pos = np.zeros(obligors_nb)
for idx, ps in zip(bc_ids, bc_positions):
    pos[idx] = ps
    
pos = pos / -pos[b_id]
    
positions = np.array(pos).reshape(pos.size, 1)

print pos

[ 0.          0.          0.          0.          0.52272727  0.20454545
  0.          0.          0.         -1.04545455  0.          0.          0.
 -0.11363636  0.          0.          0.          1.56818182  0.          0.
  0.          0.          0.          0.          0.          0.
 -0.81818182  0.          0.         -1.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.77272727  0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.         -0.09090909  0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0

In [20]:
from finance.portfolio import CCPPortfolio

port = CCPPortfolio(positions, [notional], [swap], [exposure])

# VM, IM and DF accounts

In [21]:
from ccp.accounts import Accounts, DFAccounts
from ccp.states import MembersState

states = MembersState(obligors_nb)

vm_accounts = Accounts(states, derivatives_nb)
im_accounts = DFAccounts(states, derivatives_nb) # because we need the total_amount implemented in DFAccounts 
df_accounts = DFAccounts(states, derivatives_nb)

# Skin in the game

In [22]:
from ccp.sig import SkinInTheGame

sig = SkinInTheGame()

# Regulatory capital

In [23]:
from risk.basel.exposures import BaselExposureAtDefault

ead = BaselExposureAtDefault(port, [eee])

In [24]:
from risk.basel.capital import CCPRegulatoryCapital2014

regul_cap = CCPRegulatoryCapital2014(ead, df_accounts, sig)    
epsilon_ead = 1./12
cap_conf_level = quantile_im

# Funding and capital parameters

In [25]:
k_ = 0.1
funder_recovery = 1.
c_0 = 0.0030

# Creation of the different taks

In [26]:
class ITask(object):
    level = None
    
    def perform(self, **kwargs):
        raise NotImplementedError()

In [27]:
class PutMembersAtDefaultTask(ITask):
    level = 1
    
    def __init__(self, **kwargs):
        self.__states = kwargs["states"]
        self.potential_defaulters = kwargs["defaulters"]
        self.defaulters = []
        
    def perform(self, **kwargs):
        self.__tau = kwargs["t"]
        
        for i in self.potential_defaulters:
            for j in list(i):
                if self.__states.is_alive(j):
                    self.defaulters.append(j)
                    self.__states.die(j)
            
        self.defaulters.sort()

In [28]:
class LiquidatePortfolioTask(ITask):
    level = 1
    
    def __init__(self, **kwargs):
        self.__cm_index = kwargs["bank_index"]
        
        self.__default_task = kwargs["default_task"]
                
        self.__port = kwargs['portfolio']
        self.__vm_accounts = kwargs['vm_accounts']
        self.__im_accounts = kwargs['im_accounts']
        self.__df_accounts = kwargs['df_accounts']
                
        self.__discount = kwargs["discount"]
        self.__sig = kwargs['sig']
        
        self.results = {}
        
    def perform(self, **kwargs):
        self.__tau_delta = kwargs["t"]
        
        self.new_defaulters_ids = self.__default_task.defaulters
        if not self.new_defaulters_ids:
            self.results["cva"] = 0.
            return
        
        p_tau_delta_f = lambda ii: self.__port.compute_value(self.__tau_delta, towards_=ii)
        p_tau_delta = np.array(map(p_tau_delta_f, self.new_defaulters_ids))
        p_tau_delta = np.sum(p_tau_delta, axis=1)
        
        q_tau_delta = p_tau_delta
        
        gamma_tau_delta = self.__vm_accounts.amounts + self.__im_accounts.amounts + self.__df_accounts.amounts
        gamma_tau_delta = gamma_tau_delta[self.new_defaulters_ids, :]
        gamma_tau_delta = np.sum(gamma_tau_delta, axis=1)
        
        loss_tau_delta = q_tau_delta - gamma_tau_delta
        remain_collat_loss = np.maximum(loss_tau_delta, 0.).sum()
        remaining_loss = self.__sig.handle_breach(remain_collat_loss)
        
        if remaining_loss > 0:
            frac = self.__df_accounts.get_amount(self.__cm_index).sum() / self.__df_accounts.total_default_fund().sum()
        else:
            frac = 1.0
            
        partial_loss = frac*remaining_loss
        tmp_cva = self.__discount(self.__tau_delta) * partial_loss 
        
        self.results["cva"] = tmp_cva

In [29]:
class FillVMandIMTask(ITask):
    level = 0
    
    def __init__(self, **kwargs):
        self._delta = kwargs["delta"]
        
        self._port = kwargs['portfolio']        
        self._vm_accounts = kwargs['vm_accounts']
        self._im_accounts = kwargs['im_accounts']
        
        self._quantile_im = kwargs['quantile_im']
        
        self._zeros = np.zeros(self._port.derivatives.size)
        self._zeros.fill(1e-15)
        
    def perform(self, **kwargs):
        self._tau_minus = kwargs["t"]
        
        ####################
        ## VM and IM part ##
        ####################
        for ii in xrange(self._port.counterparties_number):
            p_tau_minus_ii = self._port.compute_value(self._tau_minus, towards_=ii)
            self._vm_accounts.put_amounts(ii, p_tau_minus_ii)
            
            im_ii = self._port.compute_exposure(self._tau_minus, 
                                                risk_period=self._delta, 
                                                conf_level=self._quantile_im,
                                                towards_=ii)
            
            self._im_accounts.put_amounts(ii, np.maximum(im_ii, self._zeros).tolist())

In [30]:
class FillIMVMandDFTask(FillVMandIMTask):
    def __init__(self, **kwargs):
        super(FillIMVMandDFTask, self).__init__(**kwargs)
        self._df_accounts = kwargs['df_accounts']        
        self._ead_computer = kwargs['ead_computer']
        self._epsilon = kwargs['ead_epsilon']
        self._cover2 = kwargs['cover2rule']
                
    def perform(self, **kwargs):
        super(FillIMVMandDFTask, self).perform(**kwargs)
        self._t = kwargs["t"]
        
        eads = self._ead_computer(t=self._t,
                                 epsilon=self._epsilon, 
                                 risk_period=self._delta,
                                 alpha=self._quantile_im)
        
        total_df = self._cover2(eads)
        im_total = self._im_accounts.total_default_fund()
        
        im_proportions = self._im_accounts.amounts / im_total
        
        df_proportions = im_proportions * total_df
        
        for ii in xrange(self._port.counterparties_number):
            self._df_accounts.put_amounts(ii, df_proportions[ii, :].flatten())

In [31]:
class ReFillSigTask(ITask):
    level = 0
    
    def __init__(self, **kwargs):
        self.__sig = kwargs['sig']
        
        self.__regul_cap = kwargs['regul_capital']
        self.__cap_conf_level = kwargs['cap_conf_level']
        
        self.__delta = kwargs['delta']
        self.__port = kwargs['portfolio']
        
        self.__epsilon_ead = kwargs['epsilon_ead']
        
        self.__collat_task = FillIMVMandDFTask(delta=self.__delta, 
                                               portfolio=self.__port,
                                               vm_accounts=kwargs['vm_accounts'],
                                               im_accounts=kwargs['im_accounts'], 
                                               quantile_im=kwargs['quantile_im'],
                                               df_accounts=kwargs['df_accounts'], 
                                               ead_computer=kwargs['ead_computer'],
                                               ead_epsilon=kwargs['ead_epsilon'],
                                               cover2rule=kwargs['cover2rule'])
    
    def perform(self, **kwargs):
        t = kwargs["t"]        
        
        self.__collat_task.perform(t=t)
        
        if t == 0.:
            self.__sig.recover()
        else:
            self.__sig.update_value(t, regul_capital=self.__regul_cap, 
                                    epsilon=self.__epsilon_ead, risk_period=self.__delta,
                                    alpha=self.__cap_conf_level)

In [32]:
class ComputeXVATask(ITask):
    level = 1
    
    def __init__(self, **kwargs):
        self.__cm_index = kwargs["bank_index"]
        
        self.__delta = kwargs["delta"]
        self.__exp_distrib = kwargs["exp_distrib"]
        
        self.__copula = kwargs["copula"]
        self.__cm_subset_indexes = kwargs["bank_subsets_indexes"]
        
        self.__port = kwargs['portfolio']        
        self.__time_grid = self.__port.derivatives[0].underlying.time
        
        self.__vm_accounts = kwargs['vm_accounts']
        self.__im_accounts = kwargs['im_accounts']
        self.__df_accounts = kwargs['df_accounts']
        
        self.__discount = kwargs["discount"]
        
        self.__regul_cap = kwargs['regul_capital']
        self.__epsilon_ead = kwargs['epsilon_ead']
        self.__cap_conf_level = kwargs['cap_conf_level']
        
        self.__c_zeta = kwargs["c_zeta"]
        self.__k_zeta = kwargs["k_zeta"]
        self.__funder_recov = kwargs["funder_recovery"]
        
        self.__collat_task = FillVMandIMTask(delta=delta, portfolio=self.__port,
                                               vm_accounts=self.__vm_accounts,
                                               im_accounts=self.__im_accounts, 
                                               quantile_im=kwargs['quantile_im'])
        
        self.results = {}
    
    def perform(self, **kwargs):
        self.__zeta = kwargs["t"]
        self.__zeta_delta = time_offseter(self.__zeta+self.__delta, self.__time_grid, True) 
        
        inv_pdf = 1./self.__exp_distrib.pdf(zeta)
        
        beta_zeta = self.__discount(self.__zeta)
        
        self.__collat_task.perform(t=self.__zeta)
        
        ################
        ### DVA part ###
        ################
        p_zeta_delta = self.__port.compute_value(self.__zeta_delta, towards_=self.__cm_index).sum()
        q_zeta_delta = p_zeta_delta

        df = self.__df_accounts.get_amount(self.__cm_index).sum()
        Cstar = self.__vm_accounts.get_amount(self.__cm_index).sum()
        Cstar += self.__im_accounts.get_amount(self.__cm_index).sum()
        C = Cstar + df
        
        loss = np.maximum(q_zeta_delta - C, 0.)
        
        gamma_zeta = self.__copula.tot_gamma(self.__zeta, subsets_indexes=self.__cm_subset_indexes)
        
        self.results["dva"] = inv_pdf*self.__discount(self.__zeta_delta)*gamma_zeta*loss
        
        ################
        ### FVA part ###
        ################
        p_zeta = self.__port.compute_value(self.__zeta, towards_=self.__cm_index).sum()
        fund = p_zeta - C
        
        self.results["mla"] = inv_pdf * beta_zeta * self.__c_zeta * (-fund)
        
        bar_lambda_zeta = gamma_zeta * 0.6
        tilde_lambda_zeta = bar_lambda_zeta - (1 - self.__funder_recov) * gamma_zeta
        
        lambda_zeta = 0.
        mva = tilde_lambda_zeta * (-np.minimum(p_zeta - Cstar, 0.)) - lambda_zeta * np.maximum(p_zeta - Cstar, 0.)
        self.results["mva"] = inv_pdf * beta_zeta * mva
        
        ################
        ### KVA part ###
        ################
        kva = self.__regul_cap.compute_k_cm(self.__cm_index, 
                                            self.__zeta, 
                                            epsilon=self.__epsilon_ead, 
                                            risk_period=self.__delta,
                                            alpha=self.__cap_conf_level)
        
        self.results["kva"] = inv_pdf * self.__k_zeta * beta_zeta * (kva + df)

In [33]:
def merge_dict(actual_dict, to_add):
    for k, v in to_add.iteritems():
        if k not in actual_dict:
            actual_dict[k] = []
            
        actual_dict[k].append(v)
        
def sort_tasks_dict(a_dict):
    for k, v in a_dict.iteritems():
        v.sort(key=lambda x: x.level)

# MC loop

In [34]:
from utils import time_offseter

In [35]:
N = loaded_data["N"]
N = 10000

In [36]:
results_globs = {"cva": {"sum": 0., "sum2": 0.},
                 "dva": {"sum": 0., "sum2": 0.},
                 "mla": {"sum": 0., "sum2": 0.},
                 "mva": {"sum": 0., "sum2": 0.},
                 "kva": {"sum": 0., "sum2": 0.}}

In [37]:
from scipy.stats import expon

exp_distrib_params = loaded_data["randomization"]["distrib"]
exp_distrib = expon(loc=exp_distrib_params["loc"], scale=exp_distrib_params["scale"])

zetas = loaded_data["randomization"]["zetas"]
default_times_mat = loaded_data["credit"]["default_times"]

In [38]:
sig_time_grid = [t for t in time_grid if t.is_integer()]
df_time_grid = [t for t in time_grid if (12*t).is_integer()]

print df_time_grid
print
print sig_time_grid

[0.0, 0.083333333333333343, 0.16666666666666669, 0.25, 0.33333333333333337, 0.41666666666666669, 0.5, 0.58333333333333337, 0.66666666666666674, 0.75, 0.83333333333333337, 0.91666666666666674, 1.0, 1.1666666666666667, 1.25, 1.3333333333333335, 1.4166666666666667, 1.5, 1.5833333333333335, 1.6666666666666667, 1.75, 1.8333333333333335, 1.9166666666666667, 2.0, 2.0833333333333335, 2.25, 2.3333333333333335, 2.5, 2.5833333333333335, 2.666666666666667, 2.75, 2.8333333333333335, 2.916666666666667, 3.0, 3.0833333333333335, 3.166666666666667, 3.25, 3.3333333333333335, 3.416666666666667, 3.5, 3.5833333333333335, 3.666666666666667, 3.75, 3.8333333333333335, 3.916666666666667, 4.0, 4.166666666666667, 4.25, 4.416666666666667, 4.5, 4.666666666666667, 4.75, 4.916666666666667, 5.0]

[0.0, 1.0, 2.0, 3.0, 4.0, 5.0]


In [39]:
import time

tic = time.time()

b_df_times_indexes = [ii for ii, ind in enumerate(bc_subsets_indexes) if ind in b_subsets_indexes]

for i in range(N):
    # Resurrecting the states
    # resets to 0. the values
    # of the VMs and IMs
    states.resurrect_all()
    
    zeta = zetas[i]
    
    # We set the simulated underlying here
    # that has the all time grid with
    # mpor = 1./360
    swap.underlying = udlyings[i]    
    port.delete_cache()
    
    default_times = default_times_mat[i]
    b_min_df_time = default_times[b_df_times_indexes].min()
    bar_tau = min(b_min_df_time, maturity)
    
    df_times_dict = {
        t: PutMembersAtDefaultTask(states=states, defaulters=copula.subsets[bc_subsets_indexes[j]])
        for j, t in enumerate(default_times) if t < bar_tau
    }
    
    liq_times_dict = {
        time_offseter(t+delta, time_grid): 
        LiquidatePortfolioTask(bank_index=b_id, default_task=task, 
                               portfolio=port, discount=discount, sig=sig,
                               vm_accounts=vm_accounts, im_accounts=im_accounts, 
                               df_accounts=df_accounts)
        for t, task in df_times_dict.iteritems() if t+delta < bar_tau        
    }
    
    tau_minus_times_dict = {
        time_offseter(t-delta, time_grid, True):
        FillVMandIMTask(delta=delta, portfolio=port, 
                        vm_accounts=vm_accounts, im_accounts=im_accounts,
                        quantile_im=quantile_im)        
        for t in df_times_dict.keys()
    }
    
    dfund_times = [time_offseter(t, df_time_grid, True) for t in df_times_dict.keys()]    
    dfund_times_dict = {
        t: FillIMVMandDFTask(delta=delta, portfolio=port, 
                             vm_accounts=vm_accounts, 
                             im_accounts=im_accounts, quantile_im=quantile_im,
                             df_accounts=df_accounts, 
                             ead_computer=ead, ead_epsilon=epsilon_ead, cover2rule=cover2rule)
        for t in dfund_times
    }

    sig_times = {
        t: ReFillSigTask(delta=delta, portfolio=port, 
                         vm_accounts=vm_accounts, 
                         im_accounts=im_accounts, quantile_im=quantile_im,
                         df_accounts=df_accounts, ead_computer=ead, ead_epsilon=epsilon_ead, cover2rule=cover2rule,
                         sig=sig, regul_capital=regul_cap, 
                         epsilon_ead=epsilon_ead,
                         cap_conf_level=cap_conf_level)
        for t in sig_time_grid if t < zeta
    }

    xva_time = {
        t: ComputeXVATask(bank_index=b_id, exp_distrib=exp_distrib, 
                            copula=copula, portfolio=port, delta=delta,
                            bank_subsets_indexes=b_subsets_indexes,
                            vm_accounts=vm_accounts, 
                            im_accounts=im_accounts, quantile_im=quantile_im,
                            df_accounts=df_accounts,
                            discount=discount, 
                            regul_capital=regul_cap, epsilon_ead=epsilon_ead, cap_conf_level=cap_conf_level, 
                            funder_recovery=funder_recovery,
                            c_zeta=c_0, 
                            k_zeta=k_)
        for t in [zeta] if zeta <= bar_tau
    }
    
    current_times_dict = dict()
    all_dicts = [df_times_dict, liq_times_dict, tau_minus_times_dict, \
                 dfund_times_dict, sig_times, xva_time]
    
    for dd in all_dicts:
        merge_dict(current_times_dict, dd)
    
    sort_tasks_dict(current_times_dict)
    
    current_times = current_times_dict.keys()
    current_times.sort()
    
    for t in current_times:        
        tasks = current_times_dict[t]
        for task in tasks:            
            task.perform(t=t)
                    
    cva_, dva_, mva_, mla_, kva_ = [0. for _ in range(5)]
    for k, v in liq_times_dict.iteritems():
        cva_ += v.results["cva"]
        
    results_globs["cva"]["sum"] += cva_
    results_globs["cva"]["sum2"] += cva_**2
    
    for k, v in xva_time.iteritems():
        dva_ += v.results["dva"]
        mva_ += v.results["mva"]
        mla_ += v.results["mla"]
        kva_ += v.results["kva"]
        
    results_globs["dva"]["sum"] += dva_
    results_globs["dva"]["sum2"] += dva_**2
    
    results_globs["mva"]["sum"] += mva_
    results_globs["mva"]["sum2"] += mva_**2
    
    results_globs["mla"]["sum"] += mla_
    results_globs["mla"]["sum2"] += mla_**2
    
    results_globs["kva"]["sum"] += kva_
    results_globs["kva"]["sum2"] += kva_**2
    
toc = time.time()

# Results

In [40]:
print "Results for %d iterations (%s secs.)"%(N, toc-tic)
print

print "Used discount factor: %s"%discount
print

print "Bank index %i that belongs to the following MO copula subsets:"%b_id
for idx in b_subsets_indexes:
    print "- %s with pillars %s and intensity %s\n"%([x for x in copula.subsets[idx][0]], \
                                                     copula.pillars[idx], copula.intensities[idx])
    
print "Counterparties indexes:"
print c_ids

print "\n-----------------------------------------------------\n"

for idx in c_ids:
    print "Counterparty index %s belongs to the following MO copula subses:"%idx
    sub_indexes = copula.get_indexes_including(idx)
    for subset_idx in sub_indexes:
        print "- %s with pillars %s and intensity %s\n"%([x for x in copula.subsets[subset_idx][0]], \
                                                         copula.pillars[subset_idx], copula.intensities[subset_idx])        
    print
    
print "\n-----------------------------------------------------\n"
    
print "Derivatives:"
for d in [swap]:
    print "- %s"%d
    
print "\nPositions:"
print port.positions

print "\nConfidence level used for VM+IM: %.2f"%quantile_im
#print "Confidence level used for VM+IM+DF: %.2f"%quantile_df

Results for 10000 iterations (1363.71000004 secs.)

Used discount factor: Constant discount factor process with rate r = 0.02

Bank index 29 that belongs to the following MO copula subsets:
- [29] with pillars [3 5] and intensity [ 0.06140538  0.01950418]

- [64, 98, 99, 38, 70, 71, 104, 75, 76, 81, 78, 111, 17, 83, 120, 26, 124, 29, 31] with pillars [3 5] and intensity [ 0.0053745  0.0050889]

- [17, 26, 29, 31, 38, 50, 55, 64, 66, 70, 71, 75, 76, 78, 81, 83, 98, 99, 103, 104, 107, 111, 120, 122, 124] with pillars [3 5] and intensity [ 0.0006145  0.0086692]

- [0, 3, 4, 5, 9, 13, 17, 24, 25, 26, 27, 28, 29, 30, 31, 33, 36, 37, 38, 43, 45, 47, 50, 52, 55, 56, 57, 61, 64, 66, 70, 71, 72, 73, 75, 76, 78, 80, 81, 82, 83, 86, 87, 89, 90, 98, 99, 100, 103, 104, 107, 111, 112, 113, 114, 116, 118, 119, 120, 122, 124] with pillars [3 5] and intensity [  2.01650000e-04   9.33720000e-05]

- [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28,

In [41]:
from scipy.stats import norm

conf_level = 0.95
z_level = norm.ppf(0.5*(1+conf_level))

print "Results of the global xVA:\n"
keys = results_globs.keys()
keys.sort()

excel_results = dict()

for k in keys:
    mean_ = results_globs[k]['sum']/N 
    mod_var_ = (results_globs[k]['sum2']/N - mean_**2) / (N-1.)
    half_inter = z_level*np.sqrt(mod_var_)
    
    excel_results[k.upper()] = "%s \t %s"%(mean_*10000, half_inter / mean_)
    
    print "The %s for the bank lies in [%f, %f]"%(k.upper(), mean_-half_inter, mean_+half_inter)

Results of the global xVA:

The CVA for the bank lies in [0.000685, 0.000772]
The DVA for the bank lies in [0.000567, 0.000636]
The KVA for the bank lies in [0.001263, 0.001296]
The MLA for the bank lies in [0.000097, 0.000100]
The MVA for the bank lies in [0.000786, 0.000811]


In [42]:
file_ = open('../CCP quantile.txt', 'a')

file_.write("%s \t %s \n"%(b_id, quantile_im))

keys = excel_results.keys()
keys.sort()

for k in keys:
    file_.write("%s \t %s\n"%(k, excel_results[k]))
    
file_.write("\n")
file_.close()